In [1]:
import pandas as pd

In [2]:
lc_data = pd.read_pickle('/home/nmiranda/workspace/ztf_rapid/data/raw/rcf_marshallc_sncosmo_200114_2018classupdate_addedcv.pkl')

In [3]:
lc_data['ZTF19aamhgwm']

mjd,band,flux,fluxerr,zp,zpsys
float64,str4,float64,float64,float64,str2
58538.43899999978,p48g,0.0,134.59533125686363,25.0,ab
58538.463200000115,p48r,0.0,49.32078674468683,25.0,ab
58543.360299999826,p48r,0.0,9.572601846452761,25.0,ab
58547.46519999998,p48r,0.0,9.397882172104307,25.0,ab
58556.42169999983,p48r,346.7368504525324,28.74207970957345,25.0,ab
58558.24099999992,p48r,369.828179780267,40.87492097909591,25.0,ab
58559.35779999988,p48g,457.0881896148747,37.88944025727047,25.0,ab
58560.30200000014,p48i,366.43757464783334,33.75014787587976,25.0,ab
58562.31730000023,p48g,440.55486350655343,52.74958319052326,25.0,ab


In [12]:
lc_list = list()
idx_ztfid = dict()
ztfid_target = dict()
for idx, ztfid in enumerate(lc_data):
    this_lc = lc_data[ztfid]
    idx_ztfid[idx] = ztfid
    ztfid_target[ztfid] = this_lc.meta['classification']
    this_snn_lc = pd.DataFrame(
        {
            'FLUXCAL': this_lc['flux'],
            'FLUXCALERR': this_lc['fluxerr'],
            'FLT': this_lc['band'],
            'MJD': this_lc['mjd'],
            'SNID': idx,
        }
    )
    lc_list.append(this_snn_lc)
lc_df = pd.concat(lc_list)

In [5]:
lc_df

,FLUXCAL,FLUXCALERR,FLT,MJD,SNID
0,0.000000,22.752546,p48g,58676.1823,0
1,0.000000,22.752546,p48r,58676.2043,0
2,0.000000,24.718949,p48r,58683.2129,0
3,0.000000,25.646612,p48r,58683.2411,0
4,0.000000,35.402179,p48g,58683.2693,0
...,...,...,...,...,...
31,216.770410,23.958351,p48r,58476.4446,4577
32,175.388050,22.615371,p48r,58480.4494,4577
33,0.000000,20.371828,p48g,58480.4947,4577
34,139.315680,15.397738,p48r,58481.4406,4577


In [6]:
lc_df.to_csv('/home/nmiranda/workspace/ztf_rapid/data/interim/rcf_supernnova.csv', index_label='index')

In [10]:
predictions = pd.read_csv('/home/nmiranda/workspace/SuperNNova/Predictions_rcf_supernnova.csv').drop('Unnamed: 0', axis='columns').set_index('SNID')
predictions

,prob_class0,prob_class1,pred_class
SNID,,,
3520,0.060046,0.939954,1
3138,0.173650,0.826350,1
2440,0.240727,0.759273,1
1506,0.115462,0.884538,1
3927,0.177773,0.822227,1
...,...,...,...
2333,0.187334,0.812666,1
2340,0.098530,0.901470,1
2468,0.094872,0.905128,1


In [13]:
predictions['ztfid'] = predictions.index.map(idx_ztfid)

In [15]:
predictions = predictions.set_index('ztfid')

In [17]:
predictions['target'] = predictions.index.map(ztfid_target)

In [22]:
predictions[predictions['pred_class'] == 0]['target'].value_counts()

SN Ia              99
SN II              25
AGN                12
CV                 10
CV?                 5
SLSN-I              4
SN IIb              4
SN Ic               3
SN IIP              3
SN Ia-91T           3
SN IIn              3
SN?                 2
SN Ia 91bg-like     2
duplicate           2
AGN?                2
nova?               2
star                2
SN                  2
SN Ia 91T-like      2
Var                 1
SN Ic-BL            1
blazar              1
SN Ib/c             1
SLSN-II             1
Q                   1
SN Ia               1
SNIa                1
SN Ia-02cx          1
varstar?            1
SN Ia?              1
Name: target, dtype: int64

In [23]:
predictions[predictions['pred_class'] == 1]['target'].value_counts()

SN Ia        1444
SN II         330
AGN           223
CV            178
SN Ia-91T      67
             ... 
Duplicate       1
galaxy          1
Gap             1
unk             1
QSO?            1
Name: target, Length: 97, dtype: int64